# StarkNet Course Week 8

This week we are going to talk about oracles.

The `HashBuiltin` is needed to read and write to `@storage_var`s, because they are a hashmap.

In [14]:
%%file "Oracle.cairo"
%lang starknet

from starkware.cairo.common.cairo_builtins import HashBuiltin

Overwriting Oracle.cairo


Then we need block numbers to keep track of chain-time. There's also a timestamp call but it's unreliable as of now.

In [15]:
%%file -a "Oracle.cairo"

from starkware.starknet.common.syscalls import get_block_number

Appending to Oracle.cairo


We also need to check values within specific ranges, so for `felt`s, `assert_lt` and `unsigned_div_rem` are useful.

In [16]:
%%file -a "Oracle.cairo"

from starkware.cairo.common.math import assert_lt, unsigned_div_rem

Appending to Oracle.cairo


And for the final import, lets use some OZ functionality to make our lives easier and less error prone.

In [17]:
%%file -a "Oracle.cairo"

from openzeppelin.access.ownable.library import Ownable

Appending to Oracle.cairo


The first thing we need in an oracle is some place to store a sequence of values.

In [18]:
%%file -a "Oracle.cairo"

@storage_var
func timeseries(time: felt) -> (value: felt) {
}

Appending to Oracle.cairo


But then we also need to keep track of what the latest position in the timeseries is so we can retrieve it later on.

In [19]:
%%file -a "Oracle.cairo"
@storage_var
func latest_time() -> (time: felt) {
}

Appending to Oracle.cairo


It is also a good idea to notify offchain systems of what is happening in the Oracle, so whenever we receive new data, we could emit an event:

In [20]:
%%file -a "Oracle.cairo"

@event
func value_added(time: felt, value: felt) {
}

Appending to Oracle.cairo


Next, we need to initialize the contract during deployment:

In [21]:
%%file -a "Oracle.cairo"

@constructor
func constructor{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}(owner: felt) {
    // TODO(assignment): initialize ownership
    Ownable.initializer(owner);

    return ();
}

Appending to Oracle.cairo


And now, the core functionality is:

In [22]:
%%file -a "Oracle.cairo"

func _process_value{range_check_ptr}(original: felt) -> (processed: felt) {
    let (quotient, _) = unsigned_div_rem(original, 100);

    if (quotient == 0) {
        return (processed=original);
    }

    return (processed=100);
}

Appending to Oracle.cairo


In [23]:
%%file -a "Oracle.cairo"

@external
func ingest{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}(value: felt) {
    // TODO(assignment): check ownership
    Ownable.assert_only_owner();
    // TODO(assignment): get block number
    let (tip) = latest_time.read();
    let (time) = get_block_number();
    // TODO(assignment): check if there's already a value set for this block (missing test)
    with_attr error_message("Value already ingested at current block") {
        assert_lt(tip, time);
    }
    // TODO(assignment): bind the value within the range 0, 100
    let (processed_value) = _process_value(value);
    // TODO(assignment): write to the timeseries
    timeseries.write(time, processed_value);
    latest_time.write(time);
    // TODO(assignment): emit the `value_added` event (missing test)
    value_added.emit(time, processed_value);
    
    return ();
}

Appending to Oracle.cairo


Finally we should provide blockchain users (i.e.: other contracts) with some form of fetching the information inside the chain:

In [24]:
%%file -a "Oracle.cairo"

@view
func value_at{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}(time: felt) -> (value: felt) {
    return timeseries.read(time);
}

@view
func latest_value{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}() -> (value: felt) {
    let (tip) = latest_time.read();

    return timeseries.read(tip);
}

Appending to Oracle.cairo


Lets check our implementation:

In [25]:
!cd .. && protostar test tests/test_Oracle.cairo

16:29:48 [INFO] Collected 1 suite, and 4 test cases (0.054 s)
[PASS] tests/test_Oracle.cairo test_ingest_overflowing_value (time=0.11s, steps=440, memory_holes=20)
       pedersen_builtin=2 range_check_builtin=10
[PASS] tests/test_Oracle.cairo test_ingest_values (time=0.35s, steps=847, memory_holes=40)
       pedersen_builtin=4 range_check_builtin=20
[PASS] tests/test_Oracle.cairo test_ingest_value (time=0.11s, steps=440, memory_holes=20)
       pedersen_builtin=2 range_check_builtin=10
[PASS] tests/test_Oracle.cairo test_ingest_edge_value (time=0.11s, steps=440, memory_holes=20)
       pedersen_builtin=2 range_check_builtin=10
                                                                                [5/?]
16:29:53 [INFO] Test suites: 1 passed, 1 total5/?]
16:29:53 [INFO] Tests:       4 passed, 4 total
16:29:53 [INFO] Seed:        2127275812
16:29:53 [INFO] Execution time: 6.36 s
